## Descrição do problema (k-centros)
Dado um conjunto de $n$ pontos e um parâmetro $k$, deseja-se encontrar $k$ centros de forma que os pontos estejam o mais próximo possível desses centros (cada ponto está localizado a uma distância máxima $r$ de um dos centros).

**Entrada:** Conjunto de pontos $S = {s_1, s_2, ..., s_n}$; Função(métrica) de distância $\text{dist}: S \times S \rightarrow \mathbb{R}^+$; Inteiro (número de centros) $k$.

**Saída:** Conjunto de centros $C = {c_1, c_2, ..., c_k}$

**Objetivo:** minimizar o $r$ máximo dos clusters, $r(C) = \max \{ \text{dist}(s_i, C) \}$ onde $\text{dist}(s_i, C) = \min \{ \text{dist}(s_i, c_j) \}$


In [ ]:
# bibliotecas

import numpy as np
from ucimlrepo import fetch_ucirepo
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import silhouette_score, adjusted_rand_score
import csv
import time

In [ ]:
# datasets (1min)
absenteeism_at_work = fetch_ucirepo(id=445) 
statlog_vehicle_silhouettes = fetch_ucirepo(id=149) 
rice_cammeo_and_osmancik = fetch_ucirepo(id=545) 
raisin = fetch_ucirepo(id=850) 
wine_quality = fetch_ucirepo(id=186)
blood_transfusion_service_center = fetch_ucirepo(id=176) 
mammographic_mass = fetch_ucirepo(id=161) 
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
maternal_health_risk = fetch_ucirepo(id=863) 
yeast = fetch_ucirepo(id=110) 

In [ ]:
# Prepare the datasets
def prepare_data(dataset, label):
    X = dataset.data.features
    
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)
    
    y_true = dataset.data.targets
    
    if hasattr(y_true, 'values'):
        y_true = y_true.values.flatten()
    
    label_encoder = LabelEncoder()
    y_true_encoded = label_encoder.fit_transform(y_true)  # Encode labels as integers
    
    return X_imputed, y_true_encoded, label

real_data = [
    prepare_data(absenteeism_at_work, 'abstenteeism_at_work'),
    prepare_data(statlog_vehicle_silhouettes, 'statlog_vehicle_silhouettes'),
    prepare_data(rice_cammeo_and_osmancik, 'rice_cammeo_and_osmancik'),
    prepare_data(raisin, 'raisin'),
    prepare_data(wine_quality, 'wine_quality'),
    prepare_data(blood_transfusion_service_center, 'blood_transfusion_service_center'),
    prepare_data(mammographic_mass, 'mammographic_mass'),
    prepare_data(breast_cancer_wisconsin_original, 'breast_cancer_wisconsin_original'),
    prepare_data(maternal_health_risk, 'maternal_health_risk'),
    prepare_data(yeast, 'yeast')
]

k_values = [len(np.unique(data[1])) for data in real_data]
print(k_values)

### ***Métricas:*** Distância de Minkowski
A distância entre dois pontos $x$ e $y$ em $\mathbb{R}^n$ é dada por:

$$
\text{dist}(x, y) = \left( \sum_{i=1}^{n} |x_i - y_i|^p \right)^{\frac{1}{p}}
$$

In [ ]:
def minkowski_dist(x,y,p):
    # x, y -> tuplas
    # p -> int
    x = np.asarray(x, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    d = 0
    
    d = np.sum(np.power(np.abs(x - y), p))
    
    return np.power(d, 1 / p)

### Matrizes de distância entre os pontos 
$p=1$: Distância de Manhattan

In [ ]:
def getDistMatrix(samp, X, p):
    matrix = np.zeros((samp, samp))
    for i in range(samp):
        for j in range(i + 1, samp):
            dist = minkowski_dist(X[i], X[j], p)
            matrix[i, j] = dist
            matrix[j, i] = dist  # simetria
    return matrix

X = real_data[0][0]
samples = X.shape[0]
X_arr = X.to_numpy() if hasattr(X, 'to_numpy') else np.array(X) # convertendo o dataset pra numpy (evitar erro de indexação)
manhattan_d = getDistMatrix(samples, X_arr, p=1)
print(manhattan_d)

$p=2$: Distância Euclidiana

In [ ]:
euclid_d = getDistMatrix(samples, X_arr, p=2)
print(euclid_d)

In [ ]:
# Entrada: número N de samples, número k de centros, matriz dist de distâncias
# Saída: raio r, seguido de uma lista com os índices dos k centros escolhidos
# A escolha do primeiro centro é aleatória.

def alg_maiores_distancias(N, k, dist):
    ans = []
    ans.append(random.randrange(N))
    for i in range(k-1):
        best = -math.inf
        center = None
        for p in range(N):
            val = math.inf
            for q in ans:
                val = min(val, dist[p][q])
            if val > best:
                center = p
                best = val
        ans.append(center)

    r = -math.inf
    for p in range(N):
        best = math.inf
        for q in ans:
            best = min(best, dist[p][q])
        r = max(r, best)

    return r, ans 

In [ ]:
#   Entrada: número N de samples, número k de centros, matriz dist de distâncias,
# tamanho rmax do intervalo inicial, razão precision do tamanho do intervalo final,
# quantidade trials de tentativas para cada iteração da busca binária.
#   Saída: raio r, seguido de uma lista com os índices dos k centros escolhidos.
#   A cada iteração, a escolha dos centros é aleatória.

def alg_busca_binaria(N, k, dist, rmax = None, precision = 0.0001, trials = 100):
    if rmax == None:  # O(N^2). É melhor ter isso pré-computado
        rmax = 0
        for i in range(samples):
            for j in range(samples):
                rmax = max(rmax, euclid_d[i][j])
    lower_bound = 0
    upper_bound = rmax
    ans = []
    while upper_bound - lower_bound >= precision * rmax:
        mid = (upper_bound + lower_bound)/2
        ok = 0
        for _ in range(trials):
            centro = [False] * N
            coberto = [False] * N
            sobra = N
            for i in range(k):
                c = random.randrange(N)
                while centro[c] or (sobra > 0 and coberto[c]):
                    c = random.randrange(N)
                centro[c] = True
                for p in range(N):
                    if not coberto[p] and dist[p][c] <= mid:
                        coberto[p] = True
                        sobra -= 1
            if sobra == 0:
                ok = 1
                upper_bound = mid
                ans = []
                for i in range(N):
                    if centro[i]:
                        ans.append(i)
                break
        if not ok:
            lower_bound = mid
    return upper_bound, ans

## Dados sintéticos
### Scikit Learn
Snippet de código obtido na referência: https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html#sphx-glr-auto-examples-cluster-plot-cluster-comparison-py

In [ ]:
# Datasets creation
n_samples = 500
seed = 30

noisy_circles = datasets.make_circles(
    n_samples=n_samples, factor=0.5, noise=0.05, random_state=seed
)

noisy_moons = datasets.make_moons(n_samples=n_samples, noise=0.05, random_state=seed)

blobs = datasets.make_blobs(n_samples=n_samples, random_state=seed)

rng = np.random.RandomState(seed)

no_structure = rng.rand(n_samples, 2), np.zeros(n_samples)

random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

varied = datasets.make_blobs(
    n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=random_state
)

data = [
    noisy_circles,  
    noisy_moons,    
    varied,         
    aniso,          
    blobs,         
    no_structure,  
]

labels = [
    "noisy_circles",
    "noisy_moons",
    "varied_blobs",
    "aniso_blobs",
    "blobs",
    "no_structure"
]

labeled_data = [(X, y, label) for (X, y), label in zip(data, labels)]

# Plot
plt.figure(figsize=(9, 9))
for i, (X, y) in enumerate(data):
    X = StandardScaler().fit_transform(X)
    plt.subplot(3, 3, i + 1)
    plt.scatter(X[:, 0], X[:, 1], s=10, color="b")
    plt.title(f"Dataset {i+1}")
    plt.xlim(-2.5, 2.5)
    plt.ylim(-2.5, 2.5)
    plt.xticks(())
    plt.yticks(())

plt.tight_layout()
plt.show()


In [ ]:
colors = ['blue', 'orange', 'green', 'red']

def calculate_and_plot_2d_clusters(X, k, ax, title, p):
    X = StandardScaler().fit_transform(X)
    n_samples = X.shape[0]
    dist_matrix = getDistMatrix(n_samples, X, p)

    # alg_maiores_distancias
    r_maiores, centers_maiores = alg_maiores_distancias(n_samples, k, dist_matrix)
    labels_maiores = np.zeros(n_samples, dtype=int)
    for p in range(n_samples):
        distances_to_centers = [dist_matrix[p][center] for center in centers_maiores]
        labels_maiores[p] = np.argmin(distances_to_centers)

    # alg_busca_binaria
    r_busca, centers_busca = alg_busca_binaria(n_samples, k, dist_matrix)
    labels_busca = np.zeros(n_samples, dtype=int)
    for p in range(n_samples):
        distances_to_centers = [dist_matrix[p][center] for center in centers_busca]
        labels_busca[p] = np.argmin(distances_to_centers)

    # Plot    
    ax[0].scatter(X[:, 0], X[:, 1], c=[colors[label % len(colors)] for label in labels_maiores], s=10)
    ax[0].set_title(f"{title} - alg_1")
    ax[0].set_xlim(-2.5, 2.5)
    ax[0].set_ylim(-2.5, 2.5)
    ax[0].set_xticks(())
    ax[0].set_yticks(())

    ax[1].scatter(X[:, 0], X[:, 1], c=[colors[label % len(colors)] for label in labels_busca], s=10)
    ax[1].set_title(f"{title} - alg_2")
    ax[1].set_xlim(-2.5, 2.5)
    ax[1].set_ylim(-2.5, 2.5)
    ax[1].set_xticks(())
    ax[1].set_yticks(())

In [ ]:
plt.figure(figsize=(12, 12))

k=3
for i, (X, y) in enumerate(data):
    if X is not None:
        ax = plt.subplot(4, 4, 2 * i + 1)
        calculate_and_plot_2d_clusters(X, k, [ax, plt.subplot(4, 4, 2 * i + 2)], f"Dataset {i+1}", p=2)

plt.tight_layout()
plt.show()

### Distribuição normal multivariada

In [ ]:
def generate_datasets(dts=10, cts=3, points_center=100, desv=None):
    if desv is None:
        desv = [0.1, 0.5, 1.0]  # desvios: baixo, médio e alto
    datasets = []

    ind = 1
    for desvio in desv:
        for _ in range(dts):
            centers = np.random.rand(cts, 2) * 4 - 2
            X = []
            y = []
            for i, center in enumerate(centers):
                points = np.random.multivariate_normal(center, np.eye(2) * desvio, points_center)
                X.append(points)
                y.append(np.full(points_center, i))
            X = np.vstack(X)
            y = np.hstack(y)
            datasets.append((X, y, f"{ind}: {desvio}"))
            ind += 1
            
    return datasets

# gerando os conjuntos de dados
data_2 = generate_datasets()

# Plots
def plot_generated_datasets(datasets):
    plt.figure(figsize=(15, 10))
    for i, (X, y, title) in enumerate(datasets):
        ax = plt.subplot(5, 6, i + 1)
        cluster_colors = [colors[label % len(colors)] for label in y]
        ax.scatter(X[:, 0], X[:, 1], c=cluster_colors, s=10)
        ax.set_title(title)
        ax.set_xticks(())
        ax.set_yticks(())
    plt.tight_layout()
    plt.show()

plot_generated_datasets(data_2)

In [ ]:
n_datasets = len(data_2)
n_rows = (n_datasets * 2 + 1) // 6
plt.figure(figsize=(15, n_rows*2))

for i, (X, y, title) in enumerate(data_2):
    ax = plt.subplot(n_rows, 6, 2 * i + 1)
    calculate_and_plot_2d_clusters(X, k=3, ax=[ax, plt.subplot(n_rows, 6, 2 * i + 2)], title=title, p=2)

plt.tight_layout()
plt.show()

## Experimentos

In [ ]:
# definindo o score de silhueta quando a métrica não é euclidiana
def silhouette_score_custom(X, labels, distance_matrix):
    unique_labels = np.unique(labels)
    n_samples = len(labels)
    if len(unique_labels) == 1:
        return 0.0
    
    a = np.zeros(n_samples)
    for label in unique_labels:
        mask = (labels == label)
        cluster_points = distance_matrix[np.ix_(mask, mask)]
        a[mask] = np.mean(cluster_points, axis=1)
    
    b = np.full(n_samples, np.inf)
    for label in unique_labels:
        mask = (labels == label)
        other_labels = unique_labels[unique_labels != label]
        for other_label in other_labels:
            other_mask = (labels == other_label)
            other_cluster_points = distance_matrix[np.ix_(mask, other_mask)]
            mean_dist = np.mean(other_cluster_points, axis=1)
            b[mask] = np.minimum(b[mask], mean_dist)
    
    silhouette_scores = (b - a) / np.maximum(a, b)
    return np.mean(silhouette_scores)

# calcular métricas de qualidade
def evaluate(X, y_true, labels, metric="euclidean", matrix=None):
    if(metric == "euclidean"):
        silhouette = silhouette_score(X, labels, metric=metric)
        rand_index = adjusted_rand_score(y_true, labels)
    else:
        silhouette = silhouette_score_custom(X, labels, matrix)
        rand_index = adjusted_rand_score(y_true, labels)
        
    return silhouette, rand_index

# realizar experimentos com o algoritmo de maiores distâncias
def run_maiores_distancias(datasets, k=3, exec=30, k_list=None):
    results = []
    
    if(k_list==None):
        # dataset sintético
        for X, y_true, label in datasets:
            X = StandardScaler().fit_transform(X)
            N = len(X)
            
            for p in [1, 2]:
                dist_matrix = getDistMatrix(N, X, p)
                distance_type = "manhattan" if p == 1 else "euclidean"
                
                for _ in range(exec):
                    start_time = time.time()
                    r, centers = alg_maiores_distancias(N, k, dist_matrix)
                    exec_time = time.time() - start_time
                    
                    labels = np.argmin(dist_matrix[:, centers], axis=1)
                    
                    silhouette, rand_index = evaluate(X, y_true, labels, distance_type, dist_matrix)
                    
                    results.append([label, distance_type, r, 'Maiores Distancias', silhouette, rand_index, exec_time])
    else:
        #dataset real
        for (X, y_true, label), kx in zip(datasets, k_list):
            X = StandardScaler().fit_transform(X)
            N = len(X)

            for p in [1, 2]:
                dist_matrix = getDistMatrix(N, X, p)
                distance_type = "manhattan" if p == 1 else "euclidean"
                
                for _ in range(exec):
                    start_time = time.time()
                    r, centers = alg_maiores_distancias(N, kx, dist_matrix)
                    exec_time = time.time() - start_time
                    
                    labels = np.argmin(dist_matrix[:, centers], axis=1)
                    
                    silhouette, rand_index = evaluate(X, y_true, labels, distance_type, dist_matrix)
                    
                    results.append([label, distance_type, r, 'Maiores Distancias', silhouette, rand_index, exec_time])
                    
    return np.array(results)

# realizar experimentos com o algoritmo de variação do raio
def run_busca_binaria(datasets, exec, k_list):
    results = []
    for (X, y_true, label), kx in zip(datasets, k_list):
        X = StandardScaler().fit_transform(X)
        N = len(X)
    
        for p in [1, 2]:
            dist_matrix = getDistMatrix(N, X, p)
            distance_type = "manhattan" if p == 1 else "euclidean"
            rmax = 0
            for i in range(N):
                for j in range(N):
                    rmax = max(rmax, dist_matrix[i][j])

            for it in [2, 3, 4, 5, 6, 20]:
                prec = 2**(-it)
                
                # Estrategia 1
                for _ in range(exec):
                    start_time = time.process_time()
                    r, centers = alg_busca_binaria(N, kx, dist_matrix, rmax = rmax, precision = prec, trials = 1)
                    exec_time = time.process_time() - start_time
                    
                    labels = np.argmin(dist_matrix[:, centers], axis=1)
                    silhouette, rand_index = evaluate(X, y_true, labels, distance_type, dist_matrix)
                    results.append([label, distance_type, r, 'Busca Binaria 1', it, silhouette, rand_index, exec_time])
    
                # Estrategia 2
                start_time = time.process_time()
                r, centers = alg_busca_binaria(N, kx, dist_matrix, rmax = rmax, precision = prec, trials = 30)
                exec_time = time.process_time() - start_time
    
                labels = np.argmin(dist_matrix[:, centers], axis=1)
                silhouette, rand_index = evaluate(X, y_true, labels, distance_type, dist_matrix)
                results.append([label, distance_type, r, 'Busca Binaria 2', it, silhouette, rand_index, exec_time])
    return np.array(results)

# salvar tabelas
def save_csv(results, filename="clustering_results.csv"):
    header = ["Dataset", "Distance", "Radius", "Algorithm", "Silhouette Score", "Adjusted Rand Index", "Execution Time"]

    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(results)


### Experimentos com o k-means da scikit

In [ ]:
# encontrar raio k-means scikit
def get_radius(N, centers, labels, dist_matrix):
    max_radii = []
    
    for i in range(len(centers)):
        cluster_indices = np.where(labels == i)[0]
        max_distance = -np.inf
        for idx in cluster_indices:
            for center_idx in range(len(centers)):
                if labels[idx] == i:
                    max_distance = max(max_distance, dist_matrix[idx][center_idx])
        max_radii.append(max_distance)
    
    return max(max_radii)

# realizar experimentos com o algoritmo da scikit
def run_kmeans(datasets, k=3, exec=30, k_list=None):
    results = []
    
    if(k_list==None):
        #dataset sintético
        for X, y_true, label in datasets:
            X = StandardScaler().fit_transform(X)
            N = len(X)

            for p in [1, 2]:
                dist_matrix = getDistMatrix(N, X, p)
                distance_type = "manhattan" if p == 1 else "euclidean"
                
                for _ in range(exec):
                    start_time = time.time()
                    
                    kmeans = KMeans(n_clusters=k, random_state=0).fit(X)
                    labels = kmeans.labels_
                    exec_time = time.time() - start_time
                    centers = kmeans.cluster_centers_
                    radius = get_radius(N, centers, labels, dist_matrix)

                    
                    silhouette, rand_index = evaluate(X, y_true, labels, distance_type, dist_matrix)

                    results.append([label, distance_type, radius, 'KMeans', silhouette, rand_index, exec_time])
    else:
        #dataset real
        for (X, y_true, label), kx in zip(datasets, k_list):
            X = StandardScaler().fit_transform(X)
            N = len(X)

            for p in [1, 2]:
                dist_matrix = getDistMatrix(N, X, p)
                distance_type = "manhattan" if p == 1 else "euclidean"
                
                for _ in range(exec):
                    start_time = time.time()
                    
                    kmeans = KMeans(n_clusters=kx, random_state=0).fit(X)
                    labels = kmeans.labels_
                    exec_time = time.time() - start_time
                    centers = kmeans.cluster_centers_
                    radius = get_radius(N, centers, labels, dist_matrix)
                    silhouette, rand_index = evaluate(X, y_true, labels, distance_type, dist_matrix)

                    results.append([label, distance_type, radius, 'KMeans', silhouette, rand_index, exec_time])
    
    return np.array(results)

In [ ]:
def aggregate_experiment_results(array1, array2, csv=None):
    combined_array = np.concatenate([array1, array2])

    df = pd.DataFrame(combined_array, columns=['label', 'distance_type', 'radius', 'algorithm', 'silhouette', 'rand_index', 'exec_time'])

    df[['silhouette', 'rand_index', 'exec_time']] = df[['silhouette', 'rand_index', 'exec_time']].astype(float)

    summary_df = df.groupby(['label', 'distance_type', 'algorithm']).agg(
        silhouette_mean=('silhouette', 'mean'),
        silhouette_std=('silhouette', 'std'),
        rand_index_mean=('rand_index', 'mean'),
        rand_index_std=('rand_index', 'std'),
        exec_time_mean=('exec_time', 'mean'),
        exec_time_std=('exec_time', 'std')
    ).reset_index()
    
    if csv:
        summary_df.to_csv(csv, index=True)
        
    return summary_df

## Experimentos com os dados sintéticos

In [ ]:
# tempo de execução médio 3 min

results_md_1 = run_maiores_distancias(labeled_data)
print("Dataset 1: experimentos com algoritmo 1 concluídos")
results_kmeans_1 = run_kmeans(labeled_data)
print("Dataset 1: experimentos com k-means concluídos")
results_md_2 = run_maiores_distancias(data_2)
print("Dataset 2: experimentos com algoritmo 1 concluídos")
results_kmeans_2 = run_kmeans(data_2)
print("Dataset 2: experimentos com k-means concluídos")

In [ ]:
results_bb_1 = run_busca_binaria(labeled_data, 30, [3] * len(labeled_data))
results_bb_2 = run_busca_binaria(data_2, 30, [3] * len(data_2))

In [ ]:
def aggregate_bsearch_results(results):
    df = pd.DataFrame(results, columns=['label', 'distance_type', 'radius', 'algorithm', 'precision', 'silhouette', 'rand_index', 'exec_time'])
    
    df[['radius', 'silhouette', 'rand_index', 'exec_time']] = df[['radius', 'silhouette', 'rand_index', 'exec_time']].astype(float)
    
    summary_df = df.groupby(['label', 'distance_type', 'algorithm', 'precision']).agg(
        radius_max=('radius', 'max'),
        silhouette_mean=('silhouette', 'mean'),
        #silhouette_std=('silhouette', 'std'),
        silhouette_max=('silhouette', 'max'),
        rand_index_mean=('rand_index', 'mean'),
        #rand_index_std=('rand_index', 'std'),
        rand_index_max=('rand_index', 'max'),
        exec_time_mean=('exec_time', 'mean'),
        #exec_time_std=('exec_time', 'std'),
    ).reset_index()

    return summary_df

aggregate_bsearch_results(results_bb_1).to_csv('resultado_bsearch_sintetico_1.csv')
aggregate_bsearch_results(results_bb_2).to_csv('resultado_bsearch_sintetico_2.csv')

In [ ]:
summary_table_1 = aggregate_experiment_results(results_md_1, results_kmeans_1, csv='resultado_1.csv')
summary_2 = aggregate_experiment_results(results_md_2, results_kmeans_2)
summary_2['label'] = summary_2['label'].astype(str)
summary_2['label_first_number'] = summary_2['label'].str.extract(r'(\d+)').astype(int)
summary_table_2 = summary_2.sort_values(by='label_first_number').drop(columns=['label_first_number'])
summary_table_2.to_csv('resultado_2.csv', index=False)

## Experimentos com os dados reais

In [ ]:
results_real = run_maiores_distancias(real_data, 3, 30, k_values)
print("Dataset real: experimentos com algoritmo 1 concluídos")
save_csv(results_real, filename='cluster_real_1.csv')

results_kmeans_real = run_kmeans(real_data, 3, 30, k_values)
save_csv(results_kmeans_real, filename='k_means_real.csv')
print("Dataset real: experimentos com k-means concluídos")

# Demora uns 10 min
results_bsearch_real = run_busca_binaria(real_data, 30, k_values)
print("Dataset real: experimentos com algoritmo 2 concluídos")

In [ ]:
save_csv(results_bsearch_real, filename='bsearch_real_1.csv')

In [ ]:
summary_real = aggregate_experiment_results(results_real, results_kmeans_real, csv='resultado_real_1.csv')

In [ ]:
aggregate_bsearch_results(results_bsearch_real).to_csv('resultado_bsearch_real_1.csv')